In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install tensorflow

In [ ]:
!pip3 install torch torchvision torchaudio

In [ ]:
conda install pytorch torchvision torchaudio cpuonly -c pytorch

In [ ]:
!pip install torchsummary

In [ ]:
!pip install pandas

In [ ]:
!pip install matplotlib

In [ ]:
import os                       # for working with files
import numpy as np              # for numerical computationss
import torch                    # Pytorch module
import pandas as pd
import matplotlib.pyplot as plt # for plotting informations on graph and images using tensors
import torch.nn as nn           # for creating  neural networks
from torch.utils.data import DataLoader # for dataloaders
from PIL import Image           # for checking images
import torch.nn.functional as F # for functions for calculating loss
import torchvision.transforms as transforms   # for transforming images into tensors
from torchvision.utils import make_grid       # for data checking
from torchvision.datasets import ImageFolder  # for working with classes and images
from torchsummary import summary              # for getting the summary of our model

%matplotlib inline

In [ ]:
!pip install tensorflow

In [ ]:
!pip install scikit-learn

In [ ]:
!pip show scikit-learn

In [ ]:
!pip install opencv-python

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import tensorflow_hub as hub
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import cv2 as cv
from torch.utils.data import DataLoader

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
tf.compat.v1.losses.sparse_softmax_cross_entropy

In [ ]:
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import OneCycleLR

In [ ]:
import zipfile
import os

zip_ref = zipfile.ZipFile('/content/sample_data/Chinesetallow_whitemodel.zip','r') #Opens the zip file
zip_ref.extractall('/tmp')  #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:
# Target dimensions
root_dir = "E:/Chinesetallow_originalmodel"
target_size = (300, 300)

# Walk through the directory
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        # Check if the file is an image (you can add or remove file types as needed)
        if file.endswith('.png'):
            with Image.open(os.path.join(subdir, file)) as img:
                img_resized = img.resize(target_size)

              # Save the resized image back to the filesystem
                img_resized.save(os.path.join(subdir, f'{file}'))

    print('All images have been resized.')

In [ ]:
data_dir = "E:/Chinesetallow_originalmodel"
train_dir = data_dir + "/TRAIN"
valid_dir = data_dir + "/VALIDATE"
image_files = os.listdir(train_dir)

print (image_files)
print("Total classes are: {}".format(len(image_files)))

In [ ]:
# Initialize sets to store unique plants and colors
unique_classes = set()

# Loop through each file name and extract the plant and color
for filename in image_files:
    # Split the filename on underscores and the period before the file extension
    parts = filename.split('_')
    plants = parts[0]
    health = parts[-1].split('.')[0]

    # Add the extracted plant and color to the sets
    unique_classes.add(health)

# Print the number of unique plants and colors
print(f"Number of unique colors: {len(unique_classes)}")
print(f"Unique Plants health conditions are: \n{unique_classes}")

In [ ]:
nums = {}
for files in image_files:
    nums[files] = len(os.listdir(train_dir + '/' + files))

# converting the nums dictionary to pandas dataframe passing index as plant name and number of images as column

img_per_class = pd.DataFrame(nums.values(), index=nums.keys(), columns=["no. of images"])
img_per_class

In [ ]:
index = [n for n in range(3)] #CHANGE RANGE
plt.figure(figsize=(10, 10))
plt.bar(index, [n for n in nums.values()], width= 0.8)
plt.xlabel('Plants/Image_files', fontsize=10)
plt.ylabel('No of images available', fontsize=10)
plt.xticks(index, image_files, fontsize=10, rotation=90)
plt.title('Images per class.')

In [ ]:
n_train = 0
for value in nums.values():
    n_train += value
print(f"There are {n_train} images for training")

In [ ]:
batch_size = 20

In [ ]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize the image
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    # Normalize with the mean and standard deviation used for ResNet training
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# datasets for validation and training
train = ImageFolder(train_dir, transform=transforms.ToTensor())
valid = ImageFolder(valid_dir, transform=transforms.ToTensor())

In [ ]:
# DataLoaders for training and validation
train_dl = DataLoader(train, batch_size, shuffle=True, num_workers=2, pin_memory=True)
valid_dl = DataLoader(valid, batch_size, num_workers=2, pin_memory=True)


In [ ]:
img, label = train[0]
print(img.shape, label)

In [ ]:
# for checking some images from training dataset
def show_image(image, label):
    print("Label :" + train.classes[label] + "(" + str(label) + ")")
    plt.imshow(image.permute(1, 2, 0))

In [ ]:
show_image(*train[2400])

In [ ]:
dataset = ImageFolder('E:/Chinesetallow_originalmodel/TRAIN', transform=transform)

In [ ]:
# Create a DataLoader
data_loader = DataLoader(dataset, batch_size=20, shuffle=True)

In [ ]:
def show_batch(data_loader):
    for images, labels in data_loader:
        fig, ax = plt.subplots(figsize=(30, 30))
        ax.set_xticks([]); ax.set_yticks([])
        # Make sure to permute the dimensions to match what matplotlib expects
        ax.imshow(make_grid(images, nrow=5).permute(1, 2, 0))
        break

# Call the function with your DataLoader
show_batch(data_loader)

In [ ]:
# for moving data into GPU (if available)
def get_default_device():
    # if torch.cuda.is_available:
    #     return torch.device("cuda")
        return torch.device("cpu")

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

# for loading in the device (GPU if available else CPU)
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

device = get_default_device()
device

In [ ]:
# Moving data into CPU
train_dl = DeviceDataLoader(train_dl, device)
valid_dl = DeviceDataLoader(valid_dl, device)

In [ ]:
img_width, img_height = 300, 300

In [ ]:
# Data preprocessing and augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data preprocessing for validation
validation_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the BasicBlock
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [ ]:
# Define the ResNet class
class ResNet34(nn.Module):

    def __init__(self, block, layers, num_classes=3):
        super(ResNet34, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(512 * block.expansion, num_classes)


    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)


        return x

    def training_step(self, batch):
        images, labels = batch
        out = self(images)  # Generate predictions
        loss = F.cross_entropy(out, labels)  # Calculate loss
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)  # Generate prediction
        loss = F.cross_entropy(out, labels)  # Calculate loss
        acc = accuracy(out, labels)  # Assuming you have an accuracy function
        return {"val_loss": loss.detach(), "val_accuracy": acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x["val_loss"] for x in outputs]
        batch_accuracy = [x["val_accuracy"] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()  # Combine loss
        epoch_accuracy = torch.stack(batch_accuracy).mean()
        return {"val_loss": epoch_loss, "val_accuracy": epoch_accuracy}  # Combine accuracies

    def epoch_end(self, epoch, result):
        print(f"Epoch [{epoch}], train_loss: {result['train_loss']:.4f}, val_loss: {result['val_loss']:.4f}, val_acc: {result['val_accuracy']:.4f}")

def accuracy(outputs, labels):
        _, preds = torch.max(outputs, dim=1)
        return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
from torchsummary import summary

model = ResNet34(BasicBlock, [3, 4, 6, 3], num_classes=3)

# getting summary of the model
INPUT_SHAPE = (3, 64, 64)
print(summary(model.cpu(), (INPUT_SHAPE)))

In [ ]:
model = ResNet34(BasicBlock, [3, 4, 6, 3], num_classes=3).to(device)

In [ ]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import OneCycleLR

@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, patience=5, weight_decay=1e-4, grad_clip=0.5, opt_func=torch.optim.SGD):
    # torch.cuda.empty_cache()
    history = []

    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    sched = OneCycleLR(optimizer, max_lr=max_lr, epochs=epochs, steps_per_epoch=len(train_loader))

    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            # Gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            # Record and update learning rates
            lrs.append(get_lr(optimizer))
            sched.step()

        # Validation Phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)

#         # Early stopping logic
#         val_loss = result['val_loss']
#         if val_loss < best_val_loss:
#             best_val_loss = val_loss
#             patience_counter = 0
#         else:
#             patience_counter += 1

#         if patience_counter >= patience:
#             print("Stopping early due to no improvement in validation loss.")
#             break

    return history

In [ ]:
img, label = train[0]
print(img.shape, label)

In [ ]:
%%time
history = [evaluate(model, valid_dl)]
history

In [ ]:
model = ResNet34(BasicBlock, [3, 4, 6, 3], num_classes=3)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
train_loader = train_dl # Define your training data loader
val_loader = valid_dl # Define your validation data loader
epochs = 20
# patience = 5

In [ ]:
%%time
history = fit_one_cycle(epochs=epochs, max_lr=5e-4, model=model, train_loader=train_loader, val_loader=val_loader, weight_decay=1e-4, grad_clip=0.5, opt_func=optim.Adam)

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_accuracy'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');

def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

def plot_lrs(history):
    lrs = np.concatenate([x.get('lrs', []) for x in history])
    plt.plot(lrs)
    plt.xlabel('Batch no.')
    plt.ylabel('Learning rate')
    plt.title('Learning Rate vs. Batch no.');

In [ ]:
plot_accuracies(history)

In [ ]:
test_dir = ""
test = ImageFolder(test_dir, transform=transforms.ToTensor())

In [ ]:
test_images = sorted(os.listdir(test_dir + '/test')) # since images in test folder are in alphabetical order
test_images

In [ ]:
def predict_image(img, model):
    """Converts image to array and return the predicted class
        with highest probability"""
    # Convert to a batch of 1
    xb = to_device(img.unsqueeze(0), device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability
    _, preds  = torch.max(yb, dim=1)
    # Retrieve the class label

    return train.classes[preds[0].item()]

In [ ]:
# predicting first image
img, label = test[0]
plt.imshow(img.permute(1, 2, 0))
print('Label:', test_images[0], ', Predicted:', predict_image(img, model))

In [ ]:
# getting all predictions (actual label vs predicted)
for i, (img, label) in enumerate(test):
    print('Label:', test_images[i], ', Predicted:', predict_image(img, model))